# IRL via Apprenticeship Learning

## Overview

- https://github.com/Farama-Foundation/HighwayEnv

## Imports

In [6]:
import gymnasium as gym
import highway_env
from matplotlib import pyplot as plt
import pprint

## Environment

### Environment Creation

In [5]:
# Create the highway environment with configurations
config = {
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 4,
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": False
    },
    "action": {
        "type": "DiscreteMetaAction",
        "longitudinal": True,
        "lateral": True
    },
    "duration": 40,
    "lanes_count": 3,
    "vehicles_density": 1.0,
    "collision_reward": -1,
    "right_lane_reward": 0.1,
    "high_speed_reward": 0.4,
    "reward_speed_range": [20, 30],
    "normalize_reward": True
}

env = gym.make('highway-v0', render_mode='rgb_array', config=config)


In [7]:
import pprint
pprint.pprint(env.unwrapped.config)

{'action': {'lateral': True,
            'longitudinal': True,
            'type': 'DiscreteMetaAction'},
 'centering_position': [0.3, 0.5],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 40,
 'ego_spacing': 2,
 'high_speed_reward': 0.4,
 'initial_lane_id': None,
 'lane_change_reward': 0,
 'lanes_count': 3,
 'manual_control': False,
 'normalize_reward': True,
 'observation': {'absolute': True,
                 'features': ['presence', 'x', 'y', 'vx', 'vy'],
                 'normalize': False,
                 'type': 'Kinematics',
                 'vehicles_count': 4},
 'offroad_terminal': False,
 'offscreen_rendering': False,
 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle',
 'policy_frequency': 1,
 'real_time_rendering': False,
 'render_agent': True,
 'reward_speed_range': [20, 30],
 'right_lane_reward': 0.1,
 'scaling': 5.5,
 'screen_height': 150,
 'screen_width': 600,
 'show_trajectories': False,
 'simulation_frequency': 15,
 'vehicles_count': 50

In [10]:
# Convert to finite MDP
base_env = env.unwrapped
mdp = base_env.to_finite_mdp()

# Print environment information
print("Observation space:", mdp.observation_space)
print("Action space:", mdp.action_space)
print("Number of states:", mdp.observation_space.n)
print("Number of actions:", mdp.action_space.n)

ModuleNotFoundError: The finite_mdp module is required for conversion. No module named 'finite_mdp'

### Observation Space